# Report of Project 3: Collaboration and Competition

## Learning algorithm

The algorithm used in this project is a __Multi-Agent Deep Deterministic Policy Gradient__ (MADDPG) agent, based upon [Lowe et al.](https://arxiv.org/abs/1706.02275).

__The policies (actors)__ in MADDPG are fully-connected neural networks which output continuous values in a range of [-1, 1] for a given state input and are deterministic in nature. Therefore, to encourage exploration, especially in the beginning of training, we add __Gaussian noise__ $N_t$ to the actions generated by the neural networks, which we decay over time $t$ (episodes). That way, as with double DQN, we favor exploration in the beginning of the learning process and exploitation of the learned policy towards the end.

__The action-value functions (critics)__ are fully-connected neural networks similar to [Lillicrap et al.](https://arxiv.org/abs/1509.02971), but which differ in layer dimensions $(512, 256, 128)$ and using Leaky-ReLU activations. This architecture has been inspired from the [ddpg-bipedal](https://github.com/udacity/deep-reinforcement-learning/blob/master/ddpg-bipedal/model.py) example, provided by the authors of the Udacity's Deep Reinforcement Learning Nanodegree program.

We use the same initialization scheme as proposed by [Lillicrap et al.](https://arxiv.org/abs/1509.02971), i.e. initializing the final layers of both the actor and critic with a uniform distribution $[−3×10^{−3},3×10^{−3}]$. The upstream layers were initialized with uniform distributions $[−1 \frac{1}{√f},1 \frac{1}{√f}]$ where $f$ is the fan-in of the respective layer.

As with the double DQN and DDPG case, each experience is stored in a fixed-size __experience replay buffer__ (a double-ended queue), from which the algorithm samples batches _uniformly at random_ to fit the neural networks, breaking the temporal correlation of experiences and thus generates i.i.d. samples from the buffer, which is expected by gradient-based __optimizers__ such as with __Adam__, which is used here.

__The model-fitting process__ (i.e. update of the neural network's weights) is done every 2-nd step of the MADDPG agent, using a sampled batch of experiences. The process - for all composite agents - is similar to double DQN, which is made up by four neural networks, i.e. two online networks for actor and critic which are continuously updated, and two target networks, which are used to generate the next step's actions and Q-values respectively.

According to the algorithm of [Lowe et al.](https://arxiv.org/abs/1706.02275), we iterate over the composite agents and sample a random minibatch of $S$ samples $(\textbf{x}^j, a^j,  r^j, \textbf{x}^{'j})$ from the replay buffer, where $\textbf{x}$ is a batch of concatenated observations $o_i$  from all participating agents.

The __critic__ is optimized via the next state's estimated actions and Q-values from the target networks of both agents and the local Q-value estimated from observations, $o_i$, and actions, $a_i = \boldsymbol{\mu_{\theta_i}}(o_i) + N_t$, of all agents $i$. For the loss function, `SmoothL1Loss` a.k.a. Huber-Loss has been chosen, which is more robust to extreme values, i.e. mitigating very large gradients which potentially lead to oszillations and thus slow convergence, especially in the beginning of the learning process:

$$L(\theta_i) = \frac{1}{S}\sum_j\Big(y^j - Q^\boldsymbol{\mu}_i\big(\textbf{x}^j, a^j_1,...,a^j_N\big)\Big)^2, y^j = r^j_i+\gamma Q^\boldsymbol{\mu^{'}}_i(\textbf{x}^{'j},a^{'}_1,...,a^{'}_N)|_{a^{'}_k=\boldsymbol{\mu^{'}_k}(o^j_k)}$$

The __actor__ is optimized via maximizing the expected value of the Q-network (critic), using the states and the policies selected actions of all composite agents:

$$\nabla_{\theta_i}J \approx \frac{1}{S} \sum_j \nabla_{\theta_i}\boldsymbol{\mu}_i(o^j_i) \nabla_{a_i} Q^{\boldsymbol{\mu}}_i(\textbf{x}^j,a^j_1,...,a_i,...,a^j_N)|_{a_{i}=\boldsymbol{\mu}_i(o^j_i)}$$

The final step in the model-fitting process is to "soft-update" the target network's weights. In contrast to overwriting the target-network weights with the online network weights every N time steps, we use __Polyak Averaging__, which updates the weights more often by mixing the weights with tiny bits of both networks:

$$\theta_i^- = \tau \theta_i + (1-\tau)\theta_i^-$$

As opposed to the previous projects, we used a pretty aggressive update of the target networks, using $\tau=0.1$.

Completing this fairly concise explanation of the algorithm, the following table provides a listing of hyper-parameters used for the final results presented in the next section:

Parameter Name | Value | Description
:--- | --- | :---
actor_hidden_layer_dimensions | (128, 64) | hidden layer dimensions of the policy network.
critic_hidden_layer_dimensions | (512, 256, 128) | hidden layer dimensions of Q-network.
activation_fn (critic)| Leaky-ReLU | the activation function used for the Q-network hidden layers.
activation_fn (actor) | ReLU | the activation function used for the policy network.
buffer_size | 1000_000 | replay buffer size.
batch_size | 1024 | mini-batch size.
gamma | 0.99 | discount factor.
tau | 0.1 | interpolation parameter for target-network weight update.
lr_actor | 1e-4 | learning rate of the policy network.
lr_critic | 1e-4 | learning rate of the Q-value network.
update_every | 2 | perform optimization every N steps.
n_episodes | 5000 | maximum number of training episodes.
max_t | 1000 |  maximum number of time steps per episode.
eps_start | 1.0 | starting value of epsilon, for epsilon-greedy action selection.
eps_end | 0.01 | minimum value of epsilon.
eps_decay | 0.999 | multiplicative factor (per episode) for decreasing epsilon.
scores_window_length | 100 | length of scores window to monitor convergence.
average_target_score | 0.5 | average target score for scores_window_length at which learning stops.

## Results

The environment could be __solved in 2599 episodes__ using a local CPU environment:

* OS: macOS Big Sur (Version 11.4)
* Processors: 3,1 GHz Quad-Core Intel Core i7
* Memory: 16 GB 2133 MHz LPDDR3

![Scores](scores.png "Scores")

## Ideas for future work

From a modeling point of view, there are many ways to improve the algorithms and models even further. A (incomplete) listing of possible future improvements/extensions:

* Try Bayesian variants of the given neural network architectures (e.g. Variational Inference or Bayesian approximation using Monte-Carlo Dropout), which could possibly improve decision making by incorporating (un)certainty based on posterior-predictive distributions
* Utilize hyper-parameter optimization frameworks (hyperopt or scikit-optimize) rather than manual trial-and-error to further optimize the agent's settings

## Additional References

* [Lowe et al. "Multi-Agent Actor-Critic for Mixed Cooperative-Competitive Environments, arXiv (2020)"](https://arxiv.org/abs/1706.02275)
* [Lillicrap et al. \"Continuous Control with Deep Reinforcement Learning, ICLR (2016)\"](https://arxiv.org/abs/1509.02971)